In [1]:
import itertools
import warnings
from collections import deque
from typing import Dict

import multiprocess as mp
import pandas as pd
import scipy.stats as stats
from tqdm.notebook import tqdm

from edgedroid.models import *
from sampling_util import *

reference_name = "Offline optimum"


def experimental_run(
        rtt: float,
        proc_t: float,
        P0: float,
        Pc: float,
        task_steps: int,
        repetition: int,
):
    tc = rtt - proc_t
    w0 = 1.0
    min_sr = 1 / (2 * w0)
    alpha = 1.5

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ground_truths = {
            "low" : ExpKernelRollingTTFETModel(neuroticism=0.0),
            "high": ExpKernelRollingTTFETModel(neuroticism=1.0)
        }

        samplers: Dict[str, Dict[str, ConstantRTTSampler]] = {
            "Greedy"                           : {
                "n/a": GreedyConstantRTTSampler(
                    t_net=tc,
                    t_proc=proc_t,
                    P0=P0,
                    Pc=Pc
                )
            },
            "Wang et. al. 2019\nGaussian fit"  : {
                "n/a": JunjuesConstantRTTSampler(
                    cdf_estimator=FittedNaiveExecutionTimeModel(dist=stats.norm),
                    min_sr=min_sr,
                    alpha=alpha,
                    t_net=tc,
                    t_proc=proc_t,
                    P0=P0,
                    Pc=Pc
                )
            },
            "Sample-count-optimized\naperiodic": {
                "low" : SamplingOptimumSampler(
                    estimator=ExpKernelRollingTTFETModel(neuroticism=0.0),
                    max_wait=w0,
                    t_net=tc,
                    t_proc=proc_t,
                    P0=P0,
                    Pc=Pc
                ),
                "high": SamplingOptimumSampler(
                    estimator=ExpKernelRollingTTFETModel(neuroticism=1.0),
                    max_wait=w0,
                    t_net=tc,
                    t_proc=proc_t,
                    P0=P0,
                    Pc=Pc
                )
            },
            "Energy-optimized\naperiodic"      : {
                "low" : EnergyOptimumSampler(
                    estimator=ExpKernelRollingTTFETModel(neuroticism=0.0),
                    t_net=tc,
                    t_proc=proc_t,
                    P0=P0,
                    Pc=Pc
                ),
                "high": EnergyOptimumSampler(
                    estimator=ExpKernelRollingTTFETModel(neuroticism=1.0),
                    t_net=tc,
                    t_proc=proc_t,
                    P0=P0,
                    Pc=Pc
                )
            },
            reference_name                     : {
                "n/a": IdealConstantRTTSampler(
                    t_net=tc,
                    t_proc=proc_t,
                    P0=P0,
                    Pc=Pc
                )
            },
        }

    rows = deque()

    for (gt_level, ground_truth), (sampler_name, sampler_neuro_levels) in itertools.product(ground_truths.items(),
                                                                                            samplers.items()):
        for neuro_level, sampler in sampler_neuro_levels.items():
            ground_truth.reset()
            sampler.reset()
            prev_ttf = rtt

            for step in range(1, task_steps + 1):
                exec_time = ground_truth.advance(prev_ttf).get_execution_time()
                step_results: StepResults = sampler.sample_step(prev_ttf, exec_time)

                rows.append(dict(
                    ground_truth=gt_level,
                    sampler=sampler_name,
                    neuroticism=neuro_level,
                    repetition=repetition,
                    **step_results._asdict(),
                ))
                prev_ttf = step_results.ttf

    df = pd.DataFrame(rows)
    df["sampler"] = df["sampler"].astype(pd.CategoricalDtype(samplers.keys(), ordered=False))
    df["ground_truth"] = df["ground_truth"].astype(pd.CategoricalDtype(["low", "high"], ordered=True))
    df["neuroticism"] = df["neuroticism"].astype(pd.CategoricalDtype(["n/a", "low", "high"], ordered=True))
    return df


processing_delay = 0.250
rtts = np.linspace(0.3, 4.2, 14)
net_delays = rtts - processing_delay

task_steps = 100
repetitions = 100

P0 = 0.015
Pc = 0.045

combinations = list(itertools.product(
    rtts,
    range(1, repetitions + 1)
))

results = deque()

with tqdm(total=len(combinations)) as bar, mp.Pool() as pool:
    def _callback(df: pd.DataFrame):
        results.append(df)
        bar.update()


    def _errback(error):
        print(error)
        raise error


    for rtt, rep in combinations:
        pool.apply_async(
            experimental_run,
            args=(rtt, processing_delay, P0, Pc, task_steps, rep),
            callback=_callback,
            error_callback=_errback,
        )

    pool.close()
    pool.join()

results = pd.concat(results, ignore_index=True)
results.to_parquet("./final_sampling_results.parquet.gzip", compression="gzip")
results

In [2]:
# class SuperNeuroticModel(ExpKernelRollingTTFETModel):
#     @staticmethod
#     def get_data() -> Tuple[
#         pd.DataFrame,
#         pd.arrays.IntervalArray,
#         pd.arrays.IntervalArray,
#         pd.arrays.IntervalArray,
#     ]:
#         scaling_factor_min: float = 0.25
#         scaling_factor_max: float = 4
#
#         data, *r = ExpKernelRollingTTFETModel.get_data()
#         # print(data)
#
#         old_min = data["exec_time"].min()
#         old_max = data["exec_time"].max()
#
#         new_min = old_min * scaling_factor_min
#         new_max = old_max * scaling_factor_max
#         new_range = new_max - new_min
#
#         data["exec_time"] = (data["exec_time"] - old_min) / (old_max - old_min)
#         data["exec_time"] = (data["exec_time"] * new_range) + new_min
#
#         return data, *r
#
#
# def superneurotic_experimental_run(
#         rtt: float,
#         proc_t: float,
#         P0: float,
#         Pc: float,
#         task_steps: int,
#         repetition: int,
# ):
#     tc = rtt - proc_t
#     calc_energy = lambda result: calculate_energy(P0, Pc, tc, result)
#
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         ground_truth = SuperNeuroticModel(neuroticism=1.0)
#         samplers: Dict[str, Sampler] = {
#             "Greedy"                                     : GreedySampler(),
#             "Ideal"                                      : IdealSampler(),
#             "Moothedath, original"                       : OptimumSampler(
#                 mean_exec_time_estimator=FittedNaiveExecutionTimeModel(dist=stats.rayleigh).get_mean_execution_time,
#                 alpha_calculator=lambda: tc * (Pc - P0),
#                 beta_calculator=lambda: P0
#             ),
#             "Moothedath et. al., superneurotic estimator": OptimumSampler(
#                 mean_exec_time_estimator=SuperNeuroticModel(neuroticism=1).get_mean_execution_time,
#                 alpha_calculator=lambda: tc * (Pc - P0),
#                 beta_calculator=lambda: P0
#             )
#         }
#
#     rows = deque()
#
#     for name, sampler in samplers.items():
#         ground_truth.reset()
#         prev_ttf = rtt
#
#         for step in range(1, task_steps + 1):
#             exec_time = ground_truth.advance(prev_ttf).get_execution_time()
#             sampling_result = sampler(exec_time, rtt)
#             energy = calc_energy(sampling_result)
#
#             ttf = sampling_result.duration - exec_time
#             wait_time = ttf - rtt
#
#             rows.append({
#                 "sampler"    : name,
#                 "rtt"        : rtt,
#                 "proc_time"  : proc_t,
#                 "P0"         : P0,
#                 "Pc"         : Pc,
#                 "repetition" : repetition,
#                 "step"       : step,
#                 "exec_time"  : exec_time,
#                 "duration"   : sampling_result.duration,
#                 "ttf"        : ttf,
#                 "wait_time"  : wait_time,
#                 "energy"     : energy,
#                 "num_samples": sampling_result.num_samples,
#             })
#
#             prev_ttf = ttf
#
#     df = pd.DataFrame(rows)
#     df["sampler"] = df["sampler"].astype(pd.CategoricalDtype(samplers.keys(), ordered=False))
#     return df
#
#
# processing_delay = 0.15
# rtts = np.array([0.2, 0.4, 0.8, 1.6, 3.2, 6.4])
#
# task_steps = 100
# repetitions = 100
#
# P0 = 0.015
# Pc = 0.045
#
# combinations = list(itertools.product(
#     rtts,
#     # [processing_delay],
#     # [P0],
#     # [Pc],
#     # [task_steps],
#     range(1, repetitions + 1)
# ))
#
# results = deque()
#
# with tqdm(total=len(combinations)) as bar, mp.Pool() as pool:
#     def _callback(df: pd.DataFrame):
#         results.append(df)
#         bar.update()
#
#
#     def _errback(error):
#         print(error)
#         raise error
#
#
#     for rtt, rep in combinations:
#         pool.apply_async(
#             superneurotic_experimental_run,
#             args=(rtt, processing_delay, P0, Pc, task_steps, rep),
#             callback=_callback,
#             error_callback=_errback,
#         )
#
#     pool.close()
#     pool.join()
#
# results = pd.concat(results, ignore_index=True)
# results

In [3]:
# mean_energy_per_step = (
#     results
#     .groupby(["sampler", "rtt", "repetition"])
#     ["energy"]
#     .mean()
#     # .groupby(["sampler", "rtt"])
#     # .mean()
# )
#
# plot_data = mean_energy_per_step.reset_index()
# plot_data["rtt"] = plot_data["rtt"].apply(lambda s: f"{s:0.2f} s")
#
# fg = sns.catplot(
#     kind="bar",
#     data=plot_data,
#     x="energy",
#     y="sampler",
#     hue="rtt",
#     aspect=3,
# )
# fg.set_axis_labels("Mean energy consumed per step", "Sampler")
# fg.legend.set_title("RTT")
# for ax in fg.axes.flat:
#     ax.xaxis.set_major_formatter(tkr.FuncFormatter(lambda x, p: f"{x * 1000:1.0f} mJ"))
# plt.show()
#
# mean_energy_per_step